In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import pytensor
import pytensor.tensor as pt
import scipy

from matplotlib.lines import Line2D

In [ ]:
u_t = np.loadtxt("demo_files/u_t.txt", dtype=int)
resp = np.loadtxt("demo_files/resp.txt", dtype=int)

In [ ]:
def model(u_t, resp):
    
    omega2 =npyro.sample('omega2', dist.Normal(loc=-6, scale=20)) #
    omega3 = npyro.sample('omega3', dist.Normal(loc=-6, scale=20)) #
    kappa =npyro.sample('kappa', dist.HalfNormal(scale=20)) #

    #    omega2 = -6.115423812300319
    #    omega3 = -6.11486015098146
    #    kappa = 0.501248554597270
    

    def update(mem,#每个trial会变化的参数，mu_hat(3),pi(3)
               ur,#输入数据
               ):
        #unpack
        mu_hats, pi = mem
        mu2_t2 = mu_hats[1]
        mu3_t2 = mu_hats[2]       
        pi2_t2 = pi[1]
        pi3_t2 = pi[2]
        u_t = ur[0]
        
        ### 这里要把omega2,omega3和kappa也传进来
        omega2 = ur[1]
        omega3 = ur[2]
        kappa = ur[3]
        
        #forward
        mu_hat2_t1 = mu2_t2
        mu_hat1_t1 = 1 / ( 1 + jnp.exp(-mu_hat2_t1))

        #update  更新参数值     
        pi_hat1_t1 = 1 / (mu_hat1_t1 * (1 - mu_hat1_t1))    
        
        mu1_t1_scalar = u_t
        da1_t1 = mu1_t1_scalar - mu_hat1_t1
        
        v2_t1 = jnp.exp(kappa * mu3_t2 + omega2)
        pi_hat2_t1 = 1/((1/pi2_t2) + v2_t1 )
        # updates
        pi2_t1 = pi_hat2_t1 + 1 / pi_hat1_t1
        mu2_t1 = mu_hat2_t1 + (1 / pi2_t1) * da1_t1
        # Volatility prediction error
        da2_t1 = (1 / pi2_t1 + ((mu2_t1 - mu_hat2_t1) ** 2)) * pi_hat2_t1 - 1

        mu_hat3_t1 = mu3_t2
        pi_hat3_t1 = 1 / ((1/pi3_t2) + jnp.exp(omega3))          
        w2_t1 = v2_t1 * pi_hat2_t1

        pi3_t1 = pi_hat3_t1 + 0.5 * (kappa ** 2) * w2_t1 * ((w2_t1 + (2 * w2_t1 - 1) * da2_t1))
        mu3_t1 = mu_hat3_t1 + 0.5 * (1 / pi3_t1 ) * kappa * w2_t1 * da2_t1

        mu_hats = mu_hats.at[0].set(mu_hat1_t1)
        mu_hats = mu_hats.at[1].set(mu2_t1)        
        mu_hats = mu_hats.at[2].set(mu3_t1)
        pi = pi.at[1].set(pi2_t1)
        pi = pi.at[2].set(pi3_t1)
        
        return (mu_hats,pi), mu_hat1_t1
    
    ##input (初始化参数列表)
    mu_hats = pt.zeros([3,])
    pi = jnp.zeros([3,])
    mu_hats = mu_hats.at[0].set(0)
    mu_hats = mu_hats.at[1].set(0)        
    mu_hats = mu_hats.at[2].set(1)
    pi = pi.at[0].set(0)
    pi = pi.at[1].set(10)
    pi = pi.at[2].set(1)
    mem0 = (mu_hats, pi)
    T = len(u_t)
    
    # 这里要把参数也做成sequencex下面传进去
    om2 = jnp.ones((T,))*omega2
    om3 = jnp.ones((T,))*omega3
    kap = jnp.ones((T,))*kappa
    
    # run for loop
    (mu_hats, pi), prob = scan(update, mem0 ,(u_t, om2, om3, kap), length=T)
    
    #
    prob = npyro.sample('action', dist.Bernoulli(probs=prob), obs=resp)
   